# Story data 
In this notebook, we will sketch out a tale worth telling. This notebook will mainly be used to generate figures and data for the oultine. Every blcok of code will have some info as to why the data/figure it generates are needed.

In [2]:
import psi4
import numpy as np
from scipy.linalg import eigh
from compChem.Hartree_Fock_modes.cuhf import CUHFMolecule
from compChem.Hartree_Fock_modes.uhf import UHFMolecule
from compChem.Hartree_Fock_modes.rhf import RHFMolecule
from compChem.Hartree_Fock_modes.extensions.CIS import CISMolecule


ModuleNotFoundError: No module named 'compChem'

We will make figures of the $H_2$ stretch. These pictures will be used to show the effect of spin coontamination on a practical system.


In [ ]:
psi4.set_options({"basis":"sto-3g", "scf_type":"pk", "reference":"rhf", "d_convergence":"1e-12"}) # cc-pvdz is chosen because uhf and cuhf will use this basis as well
hydrogen = UHFMolecule("""H""")
hydrogen.setConvergence(1e-12)
data = hydrogen.iterator(mute=True)
h_energy = data[0]
h_energy

In [ ]:
import numpy as np
distances = np.arange(0.2, 5, 0.1)
energies_rhf = []
spincont_rhf = []
for distance in distances:
    h2 = RHFMolecule(f"""
    H 0 0 0
    H 0 0 {distance}
    """)
    h2.setConvergence(1e-12)
    E = h2.iterator(mute=True, criterion="energy")
    energies_rhf.append(E[0] - 2*h_energy)
    spincont_rhf.append(h2.getSpinContamination())

In [ ]:
import matplotlib.pyplot as plt
p = plt.plot(distances, energies_rhf, label="rhf energy")
plt.scatter(distances, spincont_rhf, label="spin contamination", color="black", marker=".")
plt.xlabel("distance in Angstrom")
plt.ylabel("energy in Hartree")
plt.hlines(0, 0, 5, color="red", linestyles="--")
plt.legend(loc="upper right")

In [ ]:
import numpy as np
distances = np.arange(0.2, 5, 0.1)
energies_uhf = []
spincont_uhf = []
for distance in distances:
    h2 = UHFMolecule(f"""
    H 0 0 0 
    H 0 0 {distance}
    """)
    h2.setConvergence(1e-12)
    E = h2.iterator(mute=True)
    energies_uhf.append(E[0] - 2*h_energy)
    spincont_uhf.append(h2.getSpinContamination())

In [ ]:
p = plt.plot(distances, energies_uhf, label="uhf energy")
plt.hlines(0, 0, 5, color="red", linestyles="--")
plt.scatter(distances, spincont_uhf, color="black", marker=".", label="spin contamination")
plt.xlabel("distance in Angstrom")
plt.ylabel("energy in Hartree")
plt.legend(loc="upper right")
p

In [ ]:
distances = np.arange(0.2, 5, 0.1)
energies_cuhf = []
spincont_cuhf = []
for distance in distances:
    h2 = CUHFMolecule(f"""
    H 0 0 0 
    H 0 0 {distance}
    """)
    h2.setConvergence(1e-12)
    E = h2.iterator(mute=True, criterion="energy", mixedGuess=True)
    energies_cuhf.append(E[0] - 2*h_energy)
    spincont_cuhf.append(h2.getSpinContamination())


In [ ]:
import matplotlib.pyplot as plt
p = plt.plot(distances, energies_cuhf, label="cuhf energy")
plt.hlines(0, 0, 5, color="red", linestyles="--")
plt.scatter(distances, spincont_cuhf, color="black", marker=".", label="spin contamination")
plt.xlabel("distance in Angstrom")
plt.ylabel("energy in Hartree")
plt.legend(loc="upper right")
p

In [10]:
energies = np.array([energies_rhf, energies_uhf, energies_cuhf]).T
p = plt.plot(distances, energies)
plt.hlines(0, 0, 5, color="black", linestyles="--")
plt.xlabel("distance in Angstrom")
plt.ylabel("energy in Hartree")
plt.legend(["rhf energy", "uhf energy", "cuhf energy"], loc="upper right")
plt.axis([0, 5, -0.3, 0.7])
p

ValueError: x and y must have same first dimension, but have shapes (48,) and (0, 3)

In these figures we can see that spin contamination is a very big problem in UHF. 

The CIS data still needs to be checked. We will already generate some data for $H_3$ in STO-3G.


In [7]:
h3_uhf = UHFMolecule("""
H 0 0 0
H 0 0.86602540378 0.5
H 0 0 1
units angstrom""")